In [5]:
!!pip install langchain langgraph openai pygraphviz

['Requirement already satisfied: langchain in /Users/m/Work/personal/.venv/lib/python3.13/site-packages (0.3.21)',
 'Requirement already satisfied: langgraph in /Users/m/Work/personal/.venv/lib/python3.13/site-packages (0.3.18)',
 'Requirement already satisfied: openai in /Users/m/Work/personal/.venv/lib/python3.13/site-packages (1.66.3)',
 'Collecting pygraphviz',
 '  Using cached pygraphviz-1.14.tar.gz (106 kB)',
 '  Installing build dependencies: started',
 "  Installing build dependencies: finished with status 'done'",
 '  Getting requirements to build wheel: started',
 "  Getting requirements to build wheel: finished with status 'done'",
 '  Preparing metadata (pyproject.toml): started',
 "  Preparing metadata (pyproject.toml): finished with status 'done'",
 'Requirement already satisfied: langchain-core<1.0.0,>=0.3.45 in /Users/m/Work/personal/.venv/lib/python3.13/site-packages (from langchain) (0.3.47)',
 'Requirement already satisfied: langchain-text-splitters<1.0.0,>=0.3.7 in 

In [ ]:
from typing import Dict, List, Any, Annotated, TypedDict
from langchain_openai import ChatOpenAI
from langchain.graph import Graph
from langraph.prebuilt import LanguageModel, ChatMessage
import json


ModuleNotFoundError: No module named 'langraph'

In [ ]:
# Define our state schema
class State(TypedDict):
    messages: List[Dict[str, str]]
    question: str
    coder_response: str
    critic_response: str
    turn_count: int
    max_turns: int
    complete: bool

# Initialize LLM models for our agents
def create_llm_node(system_prompt: str):
    """Create a language model node with a specified system prompt."""
    return LanguageModel(
        model=ChatOpenAI(model="gpt-3.5-turbo", temperature=0),
        system_prompt=system_prompt
    )

In [ ]:
# Define our coder agent
coder_system = """You are a coder agent. Your task is to generate Python code based on the instructions provided.
Use Thought to reason about the problem and Action to propose a code snippet.
Once you generate the code, wait for feedback from the Critic Agent and revise the code accordingly."""

# Define our critic agent
critic_system = """You are a critic agent. Your task is to review Python code and provide feedback.
Focus on clarity, correctness, and edge cases.
If the code is complete and requires no further changes, clearly state: 'The code is complete.'"""


In [ ]:
def build_reflective_code_generation_graph():
    """Build the Langraph state machine for reflective code generation."""
    
    # Define the graph
    graph = StateGraph(State)
    
    # Add nodes
    graph.add_node("coder", create_llm_node(coder_system))
    graph.add_node("critic", create_llm_node(critic_system))
    
    # Define the process functions
    def prepare_coder_input(state: State) -> Dict[str, Any]:
        """Format input for the coder agent."""
        # For initial turn, use the question
        if state["turn_count"] == 0:
            return {"messages": [{"role": "user", "content": state["question"]}]}
        # For subsequent turns, include critic feedback
        else:
            return {
                "messages": [
                    {"role": "user", "content": f"Feedback received:\n\n{state['critic_response']}\n\nPlease revise the code accordingly."}
                ]
            }
    
    def process_coder_output(state: State, output: List[ChatMessage]) -> State:
        """Process the output from the coder agent."""
        coder_response = output[0].content
        new_state = state.copy()
        new_state["coder_response"] = coder_response
        new_state["turn_count"] += 1
        
        # Check if coder wants to stop
        if "[STOP]" in coder_response:
            new_state["complete"] = True
            
        return new_state
    
    def prepare_critic_input(state: State) -> Dict[str, Any]:
        """Format input for the critic agent."""
        return {
            "messages": [
                {
                    "role": "user",
                    "content": f"The following code was generated:\n\n{state['coder_response']}\n\nPlease review and suggest improvements."
                }
            ]
        }
    
    def process_critic_output(state: State, output: List[ChatMessage]) -> State:
        """Process the output from the critic agent."""
        critic_response = output[0].content
        new_state = state.copy()
        new_state["critic_response"] = critic_response
        
        # Check if critic says the code is complete
        if "The code is complete." in critic_response:
            new_state["complete"] = True
            
        return new_state
    
    def should_continue(state: State) -> str:
        """Determine the next step based on current state."""
        # Stop if we've reached max turns or the process is complete
        if state["complete"] or state["turn_count"] >= state["max_turns"]:
            return "end"
        return "coder"
    
    # Connect the nodes in the graph
    graph.add_edge("start", "coder", prepare_coder_input)
    graph.add_edge("coder", "critic", prepare_critic_input, process_coder_output)
    graph.add_edge("critic", "conditional", process_critic_output)
    
    # Add conditional branching
    graph.add_conditional_edges(
        "conditional",
        should_continue,
        {
            "coder": "coder",
            "end": "end"
        }
    )
    
    # Compile the graph
    return graph.compile()

In [ ]:
def reflective_code_generation(question: str, max_turns: int = 5) -> str:
    """Run the reflective code generation process using Langraph."""
    # Create the Langraph state machine
    graph = build_reflective_code_generation_graph()
    
    # Initialize state
    initial_state = {
        "messages": [],
        "question": question,
        "coder_response": "",
        "critic_response": "",
        "turn_count": 0,
        "max_turns": max_turns,
        "complete": False
    }
    
    # Execute the graph
    for state, step_name in graph.stream(initial_state):
        if step_name == "coder":
            print(f"Turn {state['turn_count']}:")
            if state['turn_count'] > 0:  # Only print after first turn
                print(f"CoderAgent:\n{state['coder_response']}\n")
        
        if step_name == "critic" and not state["complete"]:
            print(f"CriticAgent:\n{state['critic_response']}\n")
            
        if state["complete"] and step_name == "critic":
            print(f" -- CriticAgent has approved the code.")
            print("[STOP]")
            
        if state["turn_count"] >= state["max_turns"] and step_name == "conditional":
            print(" -- Max turns reached.")
    
    # Return the final result
    return state["coder_response"]

In [ ]:
if __name__ == "__main__":
    prompt = """Write a Python Flask API for fetching data from MongoDB and removing the _id field before returning it."""
    result = reflective_code_generation(prompt)
    print("Resulting Output:")
    print(result)